In [2]:
from generatefeatures import generate_features
import pandas as pd
import numpy as np
import os
import rebound
from collections import OrderedDict

First we generate all the features

In [3]:
%%time
sim = rebound.Simulation.from_file('ic0000000.bin')
fullfeatures = generate_features(sim)

/mnt/raid-cita/dtamayo/stability2/lib/python2.7/site-packages/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


CPU times: user 10.2 s, sys: 0 ns, total: 10.2 s
Wall time: 10.3 s


Load the trained model

In [4]:
import pickle 
model = pickle.load(open('OptimalXGBmodel.pkl', 'rb'))

/mnt/raid-cita/dtamayo/stability2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


The trained model only uses a subset of the generated features, so make a list of the ones it needs for evaluation

In [5]:
import pandas as pd
features = []
features += ['avg_iH1', 'avg_iH2']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']
features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']
features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ['Lyapunov_time']

There's probably a better way to do this, but it seems the XGBoost API wants a dataframe passed to its predict method. So we need to turn our features into a dataframe with a single row. 

In [8]:
test = pd.DataFrame(columns=features)
test.loc[0] = fullfeatures[features] # take only the features the model uses
test

,avg_iH1,avg_iH2,norm_std_a1,norm_max_a1,norm_std_window10_a1,norm_max_window10_a1,norm_std_a2,norm_max_a2,norm_std_window10_a2,norm_max_window10_a2,...,max_ecross2,min_ecross2,avg_ecross3,std_ecross3,max_ecross3,min_ecross3,norm_a1_slope,norm_a2_slope,norm_a3_slope,Lyapunov_time
0,0.396282,0.481704,0.000329,0.001083,0.000214,0.000456,0.000701,0.001978,0.000446,0.000964,...,0.333921,0.077775,0.297364,0.016641,0.339655,0.010871,-9.963524e-11,2.144832e-10,2.681898e-11,9041.175397


Predict probability

In [9]:
model.predict_proba(test)

array([[ 0.99121308,  0.00878692]], dtype=float32)